In [17]:
import re
import pandas
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from corus import load_lenta
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# from itertools import islice
# import nltk
# nltk.download('stopwords')

Импорт данных

In [29]:
path = 'data/lenta-ru-news.csv.gz'
records = load_lenta(path)
# Считать первые N строк:
data_iterator = (next(records) for x in range(5))  # Данный вариант позволяеть сохранить исходный <class 'generator'>
# data = list(islice(records, 0, 100))
# Файл не будет читаться полностью в отличие от варианта с file.readlines().

Обработка данных

In [31]:
data_clear_list = []
for data in data_iterator:
    data_topic_lower = data.topic.lower()
    data_title_lower = data.title.lower()
    data_topic_punctuation = re.sub(r'[^\w\s]', '', data_topic_lower)  # Еще вариант составить список своих стоп слов
    data_title_punctuation = re.sub(r'[^\w\s]', '', data_title_lower)  # и проводить сравнение с ним
    data_title_stop_words = list(
        word for word in data_title_punctuation.split() if word not in stopwords.words('russian'))

    snowball = SnowballStemmer(language='russian')
    data_title_stem = list(map(snowball.stem, data_title_stop_words))

    # ...stop_words преобразован из списка в строку. str() не работает.
    # Учесть, что "".join не преобразует тип int()
    data_clear_list.append([data_topic_punctuation, " ".join(data_title_stop_words)])

print(f'len data_clear list: {len(data_clear_list)}', *data_clear_list[:5], sep='\n')

len data_clear list: 0


In [20]:
df = pandas.DataFrame(data_clear_list, columns=['topic', 'title'])
# print(df.dtypes, df.head(), df['topic'].value_counts(), sep='\n\n')

Присвоим метки для topic: Подход №1 - Найти и заменить https://clck.ru/34Nsh4

In [5]:
# topic_cat = {"topic": {"россия": 1,  "мир": 2, "спорт": 3, "путешествия": 4}}
# df = df.replace(topic_cat)
# print(topic_cat, df.dtypes, df.head(), sep='\n\n')

Присвоим метки для topic: Подход № 2 - Кодирование метки https://clck.ru/34Nsh4

In [21]:
df['topic'] = df['topic'].astype('category')
# df['topic_cat'] = df['topic'].cat.codes
df.insert(0, 'topic_cat', df['topic'].cat.codes)
# print(df.dtypes, df.head(), sep='\n\n')

Формирование обучающей и тестовой выборки

In [32]:
X_train, X_test, y_train, y_test = train_test_split(df['title'], df['topic_cat'],
                                                    train_size=0.67,
                                                    random_state=42)

print(f'len X_train: {len(X_train)}', *X_train[:5], sep='\n')
print(f'len X_test: {len(X_test)}', *X_test[:5], sep='\n')
print(f'len y_train: {len(y_train)}', *y_train[:5], sep='\n')
print(f'len y_test: {len(y_test)}', *y_test[:5], sep='\n')

len X_train: 3
обнаруж сам счастлив мест планет
назва регион росс сам высок смертност рак
сша раскр сумм расход расследован российск дел
len X_test: 2
австр представ доказательств вин российск биатлонист
хакер рассказа план великобритан заминирова севастопол
len y_train: 3
1
2
0
len y_test: 2
3
0


Векторизация документов: BOW

In [23]:
vectorizer_bow = CountVectorizer()
X_train_bow_vector = vectorizer_bow.fit_transform(X_train)
print(X_train_bow_vector)

  (0, 4)	1
  (0, 13)	1
  (0, 16)	1
  (0, 2)	1
  (0, 5)	1
  (1, 13)	1
  (1, 3)	1
  (1, 10)	1
  (1, 11)	1
  (1, 0)	1
  (1, 14)	1
  (1, 6)	1
  (2, 17)	1
  (2, 7)	1
  (2, 15)	1
  (2, 9)	1
  (2, 8)	1
  (2, 12)	1
  (2, 1)	1


Выбор модели классификатора и её обучение

In [33]:
# Построение графика для обучающего набора
fig, ax = plt.subplots(figsize=(12, 7))
# Удаление верхней и правой границ
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
# Добавление основных линий сетки
ax.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
ax.scatter(features[:-test_size]['x'], features[:-test_size]['y'], color="#8C7298")
plt.show()

NameError: name 'plt' is not defined

In [ ]:
from sklearn import svm
model = svm.SVC(kernel='poly', degree=2)
model.fit(x_train, y_train)